In [3]:
import pandas as pd
import jieba
import matplotlib.pyplot as plt

# 读取数据集，这里是直接联网读取，也可以通过下载文件，再读取
data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)

train_text = '。'.join(list(train_data[0]))
train_words = jieba.lcut(train_text)

cn_stopwords = ' '.join(pd.read_csv('https://mirror.coggle.club/stopwords/baidu_stopwords.txt', header=None)[0])
train_words = [x for x in train_words if x not in cn_stopwords]
train_words = [x for x in train_words if len(x) > 1]
train_words = [x for x in train_words if not x.isdigit()]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.651 seconds.
Prefix dict has been built successfully.


In [4]:
from collections import Counter
train_words_freq = Counter(train_words)
train_words = [x for x in train_words if train_words_freq[x] >= 5]

In [5]:
train_word_prior = {}
for row in train_data.iloc[:].itertuples():
    text, label = row[1], row[2]
    words = jieba.lcut(text)
    words = [x for x in words if x in train_words]

    if len(words) == 0:
        continue

    for word in words:
        if word not in train_word_prior:
            train_word_prior[word] = {"total": 0}

        if label not in  train_word_prior[word]:
            train_word_prior[word][label] = 0

        train_word_prior[word][label] += 1
        train_word_prior[word]["total"] += 1

In [6]:
train_word_prior = pd.DataFrame(train_word_prior).T
train_word_prior.fillna(0, inplace=True)

In [7]:
for category in train_data[1].unique():
    train_word_prior[category] /= train_word_prior['total']

In [8]:
train_word_prior['category'] = train_word_prior.columns[1:][train_word_prior.values[:, 1:].argmax(1)]
train_word_prior.groupby('category').apply(lambda x: list(x.index))

/tmp/ipykernel_59295/1433704415.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_word_prior.groupby('category').apply(lambda x: list(x.index))


category
Alarm-Update             [早上, 我定, 下午, 参加, 公司, 闹钟, 活动, 提醒, 创建, 周末, 上午, 取...
Audio-Play               [故事, 小说, 广播剧, 英文版, 岳云鹏, 爆笑, 相声, 有声, 俄语, 第五章, 郭...
Calendar-Query           [昨天, 农历, 我查, 星期, 几号, 告诉, 几月, 查查, 礼拜, 几是, 春节, 母...
FilmTele-Play            [播放, 古装, 爱情, 电视剧, 一个, 推理, 一会, 地方, 导演, 赵丽颖, 麻烦,...
HomeAppliance-Control    [空调, 客厅, 风速, 打开, 烤箱, 儿童房, 调高, 洗衣机, 停止, 工作, 模式,...
Music-Play               [随便, 一首, 专辑, 单曲, 循环, 王菲, 钢琴曲, 随机, 治愈, 日语, 歌曲, ...
Other                                     [永远, 电话, 笑话, 之间, 老婆, 不好, 漫画, 有人]
Radio-Listen             [河南, 新闻广播, 新闻台, 交通, 广播电台, 经典音乐, 七点, 中央, 电台, 都市...
TVProgram-Play           [播出, 卫视, 广西, 法治, CCTV11, 剧场, 开播, 文化, 结束, 早间, 贵...
Travel-Query             [汽车票, 回家, 深圳, 武汉, 北京, 桂林, 飞机, 起飞, 快点, 三张, 成都, ...
Video-Play               [挑战, 游戏, 视频, 和平, 精英, 花絮, 转播, 比赛, 现场, 世界, 年谍, 第...
Weather-Query            [查询, 海南, 几级, 刮风, 几天, 山西, 明天, 衡水, 气温, 适合, 杭州, 香...
dtype: object